## Su vs non

In [ ]:
import numpy as np
import pandas as pd
import pickle
import scipy
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
from sklearn.linear_model import LogisticRegression
import mp_funcs
from mp_funcs import classifiers
import plotly.express as px
import itertools
import plotly.graph_objects as go
import csv
from sklearn.model_selection import ShuffleSplit
import torch
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import auc, RocCurveDisplay, roc_curve, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_predict


### Load Data and save result to csv

In [ ]:
samples_df = pd.read_csv('samples.csv', index_col=0)
countMatrix_df = pd.read_csv('countMatrix_include_gene_name_13_samples.csv', index_col=0).T
df1_reset = countMatrix_df.reset_index(drop=True)
df2_reset = samples_df.reset_index(drop=True)
data = pd.concat([df1_reset, df2_reset], axis=1)
#data.to_csv('data.csv',  index_label=True)
data = data.groupby(by='patient').first()
data['condition']= data['condition'].apply(lambda x: 1 if x == 'Suicidal' else 0)
data.drop(columns=[ 'sex','age', 'batch','diagnosis'], inplace=True)
condition=data['condition']
condition_satisfied = data.iloc[:, :-1].loc[:, data.iloc[:, :-1].sum(axis=0) > 10]
final_data = pd.concat([condition_satisfied, data.iloc[:, -1]], axis=1)
#final_data.to_csv('final_data.csv',  index_label=True)
data = final_data.copy()
data['condition']=condition
data.to_csv('data_filterred_minimum_10_count_13_samples.csv',  index_label='patient')


# leave-one-out cross-validation (LOOCV)

In [ ]:
number_of_gene_min_p_value=50
Top_genes_in_ith_iter=20
number_of_genes_for_combination=10
numFolds=50


final_fearure_list_all=[]
search_result_all=[]


In [ ]:
for ith in range (1, 1+data.shape[0]):
#for ith in range (1, 4):
    df_without_ith_row = data.drop(ith)
    # Calculate mann-whitney u-test for eact gene and keep p-value
    stat_result = scipy.stats.mannwhitneyu(df_without_ith_row[df_without_ith_row['condition']==0] , df_without_ith_row[df_without_ith_row['condition']==1] )
    # take the 50 genes with lowest p-value
    min_pval_idx = np.argsort(stat_result[1])[:number_of_gene_min_p_value]
    focus_features_list = list(df_without_ith_row.columns[min_pval_idx])
    focus_features_list.remove('condition')
    # make sure, condition is not part of features list
    
    
    print('condition' in focus_features_list)
    norm_count_df= data.iloc[:, min_pval_idx[:number_of_gene_min_p_value]].melt(id_vars='condition',  var_name='Genes')
    norm_count_df['value']=np.log(norm_count_df['value'])


    search_result=[]
    pool = mp.Pool(processes=32)
    for k, num_of_features in enumerate(range(1, number_of_genes_for_combination)): 
        #print(k)
        for features in itertools.combinations(focus_features_list[:number_of_gene_min_p_value],num_of_features): 
        # print(features)
            score_mp = pool.apply_async(mp_funcs.global_predicat, args=(df_without_ith_row, features,))
            search_result.append({'num_of_features': num_of_features, 'features': features, 
                                  'score_mp': score_mp})

    pool.close()
    pool.join() 
    print('Done')

    search_result_all.append(search_result)

    df = pd.DataFrame(search_result)
    df['score']=df['score_mp'].apply(lambda x: x.get())



    
    groups_of_genes = df[(df['score']>0.95)]['features'].to_list()

    final_fearure_list = [x[0] for x in Counter(list(sum(groups_of_genes, ()))).most_common(Top_genes_in_ith_iter)]
    final_fearure_list_all.append(final_fearure_list)
    #plt.plot([mp_funcs.global_predicat(df_without_ith_row, final_fearure_list[:x],) for x in range(1,Top_genes_in_ith_iter+1)])


In [ ]:


# Flatten the list of lists
all_names = [name for sublist in final_fearure_list_all for name in sublist]

# Count occurrences of each name
name_counts = Counter(all_names)

# Get the 50 most common names
most_common_names = name_counts.most_common(50)

#print("50 most common names and their counts:")
#for name, count in most_common_names:
    #print(name, "-", count)
most_common_genes_list = [name for name, count in most_common_names]



In [ ]:

df = pd.DataFrame(most_common_genes_list[:20], columns=['Feature'])
df.to_csv('most_common_50_genes_list_from_13_iteration.csv', index=False)

In [ ]:
filtered_data = data[['condition'] + most_common_genes_list[:50]]
stat_result = scipy.stats.mannwhitneyu(filtered_data[filtered_data['condition']==0] , filtered_data[filtered_data['condition']==1] )
# take the 50 genes with lowest p-value
min_pval_idx = np.argsort(stat_result[1])[:50]
focus_features_list = list(filtered_data.columns[min_pval_idx])
focus_features_list.remove('condition')

'condition' in focus_features_list

In [ ]:
norm_count_df= filtered_data.iloc[:, min_pval_idx[:31]].melt(id_vars='condition',  var_name='Genes')
norm_count_df['value']=np.log(norm_count_df['value']+1)

In [ ]:
# plot 20th genes with lowest p-value ...
fig = px.strip(norm_count_df, x="Genes", y="value", color="condition",
                 title="",
                 labels={"value":"Log-Count (Arb. unit)"} # customize axis label
                )

fig.show()
fig = px.bar(x=focus_features_list[:30], y=stat_result[1][min_pval_idx[1:31]], title='Lowest Mann-Whitney UTest p-value',  )
fig.update_layout(
    xaxis_title="Genes",yaxis_title="P-Value")

fig.show()

In [ ]:
final_fearure_list=most_common_genes_list

## selecting 10 genes out of most 20 genes

In [ ]:
search_result=[]
pool = mp.Pool(processes=32)
for k, num_of_features in enumerate(range(1, 10)): 
    #print(k)
    for features in itertools.combinations(most_common_genes_list[:20],num_of_features): 
        # print(features)
        score_mp = pool.apply_async(mp_funcs.global_predicat, args=(data, features,))
        search_result.append({'num_of_features': num_of_features, 'features': features, 
                              'score_mp': score_mp})

pool.close()
pool.join() 
print('Done')

In [ ]:
#plt.plot([mp_funcs.global_predicat(data, final_fearure_list[:x],) for x in range(1,11)])

In [ ]:
df = pd.DataFrame(search_result)
df['score']=df['score_mp'].apply(lambda x: x.get())

In [ ]:
featVsScore =df.groupby('num_of_features')
plt.errorbar(x=featVsScore['score'].mean().index, y=featVsScore['score'].mean(), yerr=featVsScore['score'].std().to_list())

In [ ]:
from collections import Counter
groups_of_genes = df[(df['score']>0.95)]['features'].to_list()
with open('SuicidevsNon_suicidal_20commonGeneList_for_ML.txt','w') as f: 
    f.writelines([x[0]+'\n' for x in Counter(list(sum(groups_of_genes, ()))).most_common(20)])


In [ ]:
final_fearure_list=[x[0]+'\n' for x in Counter(list(sum(groups_of_genes, ()))).most_common(20)]


## ML training and testing on 13 samples 

In [ ]:
final_fearure_list=most_common_genes_list


In [ ]:
featuresList = final_fearure_list[:1]
scorePerClf={}
for clf_name, clf in classifiers:
    scorePerClf[clf_name]=[]
numFolds=50

ss = ShuffleSplit(n_splits=numFolds, train_size=0.50, random_state=None)
for train_index, test_index in ss.split(df_without_ith_row):

    
    # train subbet
    X_train = np.array(data.iloc[train_index,:].loc[:, featuresList])
    y_train= np.array(data.iloc[train_index,:].loc[:,'condition'])      
    
    X_test = np.array(data.iloc[test_index,:].loc[:, featuresList])
    y_test= np.array(data.iloc[test_index,:].loc[:,'condition'])
    
    for clf_name, clf in classifiers:
        
        pipe = Pipeline([  ('scaler', StandardScaler()),  ('clf', clf),  ])  
        pipe.fit(X_train,y_train)
        scorePerClf[clf_name].append(pipe.score(X_test,y_test))


scorePerClf_df= pd.DataFrame(scorePerClf)
scorePerClf_df

In [ ]:
row_averages = scorePerClf_df.mean(axis=0)
row_averages

In [ ]:
fig = px.box(data_frame=scorePerClf_df.melt(value_vars=scorePerClf_df.columns, var_name='Classifier', value_name='Accuracy'), 
              y='Accuracy', x= 'Classifier', color='Classifier')
fig.update_layout(yaxis_range=[0.5,1.05], height=600)
#fig.write_image('Suicidal_Vs_Nonsuicida__Accuracy.eps')
fig.show()
for k,v in scorePerClf.items(): 
    print(f' {k}: {np.mean(v):.3f} ± {np.std(v):.3f}')

In [ ]:
featuresList = final_fearure_list[:8]
scorePerClf={}
for clf_name, clf in classifiers:
    scorePerClf[clf_name]=[]
numFolds=50

ss = ShuffleSplit(n_splits=numFolds, train_size=0.50, random_state=None)
for train_index, test_index in ss.split(df_without_ith_row):

    
    # train subbet
    X_train = np.array(data.iloc[train_index,:].loc[:, featuresList])
    y_train= np.array(data.iloc[train_index,:].loc[:,'condition'])      
    
    X_test = np.array(data.iloc[test_index,:].loc[:, featuresList])
    y_test= np.array(data.iloc[test_index,:].loc[:,'condition'])
    
    for clf_name, clf in classifiers:
        
        pipe = Pipeline([  ('scaler', StandardScaler()),  ('clf', clf),  ])  
        pipe.fit(X_train,y_train)
        scorePerClf[clf_name].append(pipe.score(X_test,y_test))


scorePerClf_df= pd.DataFrame(scorePerClf)
scorePerClf_df

In [ ]:
row_averages = scorePerClf_df.mean(axis=0)
row_averages

In [ ]:
fig = px.box(data_frame=scorePerClf_df.melt(value_vars=scorePerClf_df.columns, var_name='Classifier', value_name='Accuracy'), 
              y='Accuracy', x= 'Classifier', color='Classifier')
fig.update_layout(yaxis_range=[0.5,1.05], height=600)
fig.write_image('Suicidal_Vs_Nonsuicida__Accuracy_for_8_features.png')
#fig.write_image('Suicidal_Vs_Nonsuicida__Accuracy_for_8_features.eps')
fig.show()
for k,v in scorePerClf.items(): 
    print(f' {k}: {np.mean(v):.3f} ± {np.std(v):.3f}')

# ROC

In [ ]:
featuresList = final_fearure_list[:8]

In [ ]:
X = np.array(data.loc[:, featuresList])
y = np.array(data.loc[:,'condition'])    

In [ ]:
def plot_ROC(classifier, numFolds=5, ax=None): 
    clf = classifier[1]
    clf_label = classifier[0]
   
    tprs = []
    aucs = []
    actual_condition = np.empty([0], dtype=int)
    predicated_condition = np.empty([0], dtype=int)

    mean_fpr = np.linspace(0, 1, 100)
    cv = ShuffleSplit(n_splits=numFolds, train_size=0.5, random_state=1234).split(X,y)
    if not ax: 
        fig, ax = plt.subplots(figsize=(6, 6))
    for fold, (train, test) in enumerate(cv):
        clf.fit(X[train], y[train])
        if isinstance(clf, SVC):
            scaler = StandardScaler()
            scaler.fit(X[train])
            X[train]=scaler.transform(X[train])
            X[test]=scaler.transform(X[test])
            
            probas_ = clf.fit(X[train], y[train]).decision_function(X[test])
            # Compute ROC curve and area the curve
            fpr, tpr, thresholds = roc_curve(y[test], probas_)     
            

        else: 
            probas_ = clf.fit(X[train], y[train]).predict_proba(X[test])
            # Compute ROC curve and area the curve
            fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
            
        predicated_condition = np.append(predicated_condition, clf.predict(X[test]))
        actual_condition = np.append(actual_condition,y[test])
        
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(auc(fpr, tpr))

    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm\sigma$",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"{clf_label}\nROC curve with cross-validation",
    )
    ax.axis("square")
    ax.legend(loc="lower right")
    # plt.show()
    return ax, (mean_fpr, mean_tpr), (predicated_condition, actual_condition)

In [ ]:


def plot_confusion_matrix(actual_classes , predicted_classes , labels , ax):

    matrix = confusion_matrix(actual_classes, predicted_classes)
    
    plt.figure() #figsize=(12.8,6))
    sns.heatmap(matrix, annot=True, xticklabels=labels, yticklabels=labels, cmap="Blues", fmt="g", ax=ax)
    ax.set_xlabel('Predicted'); ax.set_ylabel('Actual'); 
    ax.title.set_text('Confusion Matrix')


    return ax

In [ ]:
rocPerClf={}
for classifier in classifiers[:]: 
    fig, axes = plt.subplots(1,2, figsize=(15,7))
    _, rocPerClf[classifier[0]], (predicated_condition, actual_condition) = plot_ROC(classifier, ax = axes[0], numFolds=50)
    plot_confusion_matrix(actual_condition , predicated_condition , labels=['Suicide', 'Non-Suicide'] , ax=axes[1])
    fig.savefig(f'Suicidal_Vs_Nonsuicida_ROC_{classifier[0]}.png')
    #fig.savefig(f'SuicidevsNon-suicide_ROC_{classifier[0]}.eps')
    #fig.savefig(f'SuicidevsNon-suicide_ROC_{classifier[0]}.pdf')
    # plt.show()

    
    

### Joined ROC

In [ ]:

# Create traces
fig = go.Figure()

for k, v in rocPerClf.items(): 
    fig.add_trace(go.Scatter(x=v[0], y=v[1],
                        mode='lines',
                        name=k))
    
fig.add_trace(go.Scatter(x=[0,1], y=[0,1],
                         line=dict(color='black', width=2,
                              dash='dash'), 
                        mode='lines',name='chance level (AUC = 0.5)'))
fig.update_layout(title='ROC mean value', width = 800, height= 600)
#fig.savefig('Suicidal_Vs_Nonsuicida_ROC_All_classifier.png')
fig.write_image('Suicidal_Vs_Nonsuicida_ROC_All_classifiers.png')
#fig.write_image('LRvsLR_ROC.eps')
fig.show()

# Testing on unklnown samples

### Load Data

In [ ]:
samples_df = pd.read_csv('samples20sample.csv', index_col=0)
countMatrix_df = pd.read_csv('countMatrix_include_gene_name20sample.csv', index_col=0).T
df1_reset = countMatrix_df.reset_index(drop=True)
df2_reset = samples_df.reset_index(drop=True)
data = pd.concat([df1_reset, df2_reset], axis=1)
#data.to_csv('data20samples.csv',  index_label=True)
data = data.groupby(by='patient').first()
data['condition']= data['condition'].apply(lambda x: 1 if x == 'Suicidal' else 0)
data.drop(columns=[ 'sex','age', 'batch','diagnosis'], inplace=True)
condition=data['condition']
condition_satisfied = data.iloc[:, :-1].loc[:, data.iloc[:, :-1].sum(axis=0) > 10]
final_data = pd.concat([condition_satisfied, data.iloc[:, -1]], axis=1)
#final_data.to_csv('final_data20samples.csv',  index_label=True)
data = final_data.copy()
data['condition']=condition
data.to_csv('data_filterred_minimum_10_count_20_samples.csv',  index_label='patient')


In [ ]:
featuresList = final_fearure_list[:8]


In [ ]:
train_index=[0,1,2,3,4,5,6,7,8,9,10,11,12]
test_index=[13,14,15,16,17,18,19]
train_index,test_index

In [ ]:
X_train = np.array(data.iloc[train_index,:].loc[:, featuresList])
y_train= np.array(data.iloc[train_index,:].loc[:,'condition'])
X_test = np.array(data.iloc[test_index,:].loc[:, featuresList])
y_test= np.array(data.iloc[test_index,:].loc[:,'condition'])

X_train,y_train,X_test,y_test

In [ ]:
#featuresList = final_fearure_list[:1]
#print(featuresList )
scorePerClf={}
PredictionPerClf={}
for clf_name, clf in classifiers:
    scorePerClf[clf_name]=[]
    PredictionPerClf[clf_name]=[]
for i in range (1,10):
    featuresList = final_fearure_list[:i]
    X_train = np.array(data.iloc[train_index,:].loc[:, featuresList])
    y_train= np.array(data.iloc[train_index,:].loc[:,'condition'])      
    
    X_test = np.array(data.iloc[test_index,:].loc[:, featuresList])
    y_test= np.array(data.iloc[test_index,:].loc[:,'condition'])
    
    for clf_name, clf in classifiers:
        
        pipe = Pipeline([  ('scaler', StandardScaler()),  ('clf', clf),  ])  
        pipe.fit(X_train,y_train)
        scorePerClf[clf_name].append(pipe.score(X_test,y_test))
        PredictionPerClf[clf_name].append(pipe.predict(X_test))


In [ ]:
for clf_name, clf in classifiers:
        
    a=torch.tensor(PredictionPerClf[clf_name]).T
    file_name_last_etr_model=clf_name+str('_prediction')+str('.csv')
    pd.DataFrame(torch.tensor(PredictionPerClf[clf_name]).T).to_csv( file_name_last_etr_model)
    print(clf_name)




In [ ]:

scorePerClf={}
PredictionPerClf={}
for clf_name, clf in classifiers:
    scorePerClf[clf_name]=[]
    PredictionPerClf[clf_name]=[]
i=8
featuresList = final_fearure_list[:i]
X_train = np.array(data.iloc[train_index,:].loc[:, featuresList])
y_train= np.array(data.iloc[train_index,:].loc[:,'condition'])      
    
X_test = np.array(data.iloc[test_index,:].loc[:, featuresList])
y_test= np.array(data.iloc[test_index,:].loc[:,'condition'])
    
for clf_name, clf in classifiers:
        
    pipe = Pipeline([  ('scaler', StandardScaler()),  ('clf', clf),  ])  
    pipe.fit(X_train,y_train)
    scorePerClf[clf_name].append(pipe.score(X_test,y_test))
    PredictionPerClf[clf_name].append(pipe.predict(X_test))


In [ ]:
for clf_name, clf in classifiers:
        
    a=torch.tensor(PredictionPerClf[clf_name]).T
    file_name_last_etr_model=clf_name+str('_prediction')+str('.csv')
    pd.DataFrame(torch.tensor(PredictionPerClf[clf_name]).T).to_csv( file_name_last_etr_model)
    print(clf_name)


